In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report, confusion_matrix
import time
import copy

# 设置随机种子确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 配置参数
DATA_DIR = r"D:\homework\论文\论文\project\code\训练\dataset_split"
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
NUM_WORKERS = 4

# 模型配置（替换VGG16为ShuffleNet v2）
MODELS = {
    "efficientnet-b0": {
        "model": models.efficientnet_b0,
        "pretrained": True,
        "feature_size": 1280,
        "classifier": lambda feat_size, num_classes: nn.Linear(feat_size, num_classes)
    },
    "resnet50": {
        "model": models.resnet50,
        "pretrained": True,
        "feature_size": 2048,
        "classifier": lambda feat_size, num_classes: nn.Linear(feat_size, num_classes)
    },
    "shufflenet_v2": {
        "model": models.shufflenet_v2_x1_0,
        "pretrained": True,
        "feature_size": 1024,
        "classifier": lambda feat_size, num_classes: nn.Linear(feat_size, num_classes)
    },
    "mobilenet_v3": {
        "model": models.mobilenet_v3_large,
        "pretrained": True,
        "feature_size": 960,  # 修正特征维度
        "classifier": lambda feat_size, num_classes: nn.Sequential(
            nn.Linear(feat_size, 1280),
            nn.Hardswish(),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1280, num_classes)
        )
    }
}

# 检查GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def create_data_transforms():
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(p=0.1),
            transforms.RandomRotation(30),
            transforms.ColorJitter(
                brightness=0.3,
                contrast=0.3,
                saturation=0.3,
                hue=0.1
            ),
            transforms.RandomGrayscale(p=0.1),
            # transforms.RandomErasing(p=0.2, scale=(0.02, 0.2)),
            transforms.RandomAffine(
                degrees=0,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1),
                shear=10
            ),
            transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def create_data_loaders(data_dir):
    """创建数据加载器"""
    data_transforms = create_data_transforms()
    
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                      for x in ['train', 'val', 'test']}
    
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                 shuffle=True if x == 'train' else False,
                                 num_workers=NUM_WORKERS)
                   for x in ['train', 'val', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes
    
    return dataloaders, dataset_sizes, class_names

def initialize_model(model_name, num_classes):
    """初始化预训练模型"""
    model_info = MODELS[model_name]
    model = model_info["model"](pretrained=model_info["pretrained"])
    
    # 修改分类器
    if model_name == "resnet50":
        model.fc = model_info["classifier"](model_info["feature_size"], num_classes)
    elif model_name == "shufflenet_v2":
        model.fc = model_info["classifier"](model_info["feature_size"], num_classes)
    elif model_name == "efficientnet-b0":
        model.classifier[1] = model_info["classifier"](model_info["feature_size"], num_classes)
    elif model_name == "mobilenet_v3":
        model.classifier = model_info["classifier"](model_info["feature_size"], num_classes)
    
    # 打印模型结构（可选）
    print(f"\n{model_name} Classifier:")
    print(model.classifier if hasattr(model, 'classifier') else model.fc)
    
    return model.to(device)

def count_parameters(model):
    """计算模型参数量（MB）"""
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return params * 4 / (1024 * 1024)  # 转换为MB

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, model_name="model"):
    """训练模型并记录指标"""
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # 记录训练历史
    history = {
        'train_loss': [], 'train_acc': [], 'train_precision': [], 'train_recall': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_precision': [], 'val_recall': [], 'val_f1': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 训练模式
            else:
                model.eval()   # 评估模式
            
            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []
            
            # 迭代数据
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 零梯度
                optimizer.zero_grad()
                
                # 前向传播
                # 只有在训练时才跟踪历史
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # 只有在训练阶段才进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            # 计算epoch指标
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            # 计算分类报告
            report = classification_report(all_labels, all_preds, output_dict=True, zero_division=0)
            macro_precision = report['macro avg']['precision']
            macro_recall = report['macro avg']['recall']
            macro_f1 = report['macro avg']['f1-score']
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} '
                  f'Precision: {macro_precision:.4f} Recall: {macro_recall:.4f} F1: {macro_f1:.4f}')
            
            # 记录历史
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            history[f'{phase}_precision'].append(macro_precision)
            history[f'{phase}_recall'].append(macro_recall)
            history[f'{phase}_f1'].append(macro_f1)
            
            # 深拷贝模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), f"{model_name}_best_model.pth")
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model, history

def evaluate_model(model, dataloader, class_names, model_name):
    """评估模型"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # 打印分类报告
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    print(f"\n{model_name} Test Metrics:")
    print(f"Accuracy: {report['accuracy']:.4f}")
    print(f"Macro Precision: {report['macro avg']['precision']:.4f}")
    print(f"Macro Recall: {report['macro avg']['recall']:.4f}")
    print(f"Macro F1-Score: {report['macro avg']['f1-score']:.4f}")
    
    # 计算混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    
    return {
        'accuracy': report['accuracy'],
        'precision': report['macro avg']['precision'],
        'recall': report['macro avg']['recall'],
        'f1': report['macro avg']['f1-score'],
        'confusion_matrix': cm
    }

def plot_training_curves(histories, metrics, save_path):
    """绘制训练曲线"""
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metrics):
        plt.subplot(2, 3, i+1)
        
        for model_name, history in histories.items():
            plt.plot(history[f'train_{metric}'], label=f'{model_name} Train')
            plt.plot(history[f'val_{metric}'], label=f'{model_name} Val')
        
        plt.title(f'Model {metric.capitalize()}')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def compare_models_table(model_metrics, model_sizes):
    """生成模型对比表格"""
    print("\n===== Model Comparison =====")
    print(f"{'Model':<15} {'Accuracy(%)':<12} {'Precision(%)':<12} {'Recall(%)':<12} {'F1-Score(%)':<12} {'Params(MB)':<12}")
    print("-" * 75)
    
    for model_name, metrics in model_metrics.items():
        print(f"{model_name:<15} {metrics['accuracy']*100:<12.2f} {metrics['precision']*100:<12.2f} "
              f"{metrics['recall']*100:<12.2f} {metrics['f1']*100:<12.2f} {model_sizes[model_name]:<12.2f}")

def main():
    """主函数"""
    # 创建数据加载器
    dataloaders, dataset_sizes, class_names = create_data_loaders(DATA_DIR)
    print(f"Classes: {class_names}")
    num_classes = len(class_names)
    
    all_histories = {}
    all_metrics = {}
    model_sizes = {}
    
    # 训练所有模型
    for model_name in MODELS.keys():
        print(f"\n===== Training {model_name} model =====")
        
        # 初始化模型
        model = initialize_model(model_name, num_classes)
        model = model.to(device)
        
        # 计算模型大小
        model_size = count_parameters(model)
        model_sizes[model_name] = model_size
        print(f"{model_name} Parameters: {model_size:.2f} MB")
        
        # 设置损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # 训练模型
        model, history = train_model(
            model, dataloaders, criterion, optimizer,
            num_epochs=EPOCHS, model_name=model_name
        )
        
        # 评估模型
        print(f"\nEvaluating {model_name} model:")
        metrics = evaluate_model(model, dataloaders['test'], class_names, model_name)
        all_metrics[model_name] = metrics
        
        # 保存训练历史
        all_histories[model_name] = history
    
    # 绘制训练曲线
    plot_training_curves(
        all_histories, 
        metrics=['loss', 'acc', 'precision', 'recall', 'f1'],
        save_path='training_curves_comparison.png'
    )
    
    # 生成模型对比表格
    compare_models_table(all_metrics, model_sizes)
    
    print("\n===== Training and evaluation completed for all models =====")
    print("Training curves saved to 'training_curves_comparison.png'")

if __name__ == "__main__":
    main()

d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0
Classes: ['Bacterial Leaf Blight', 'Brown Spot', 'Healthy Rice Leaf', 'Leaf Blast', 'Leaf scald', 'Narrow Brown Leaf Spot', 'Neck Blast', 'Rice Hispa', 'Sheath Blight']

===== Training efficientnet-b0 model =====

efficientnet-b0 Classifier:
Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=9, bias=True)
)


d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


efficientnet-b0 Parameters: 15.33 MB
Epoch 0/29
----------
train Loss: 0.9182 Acc: 0.6795 Precision: 0.6755 Recall: 0.6778 F1: 0.6745
val Loss: 0.4494 Acc: 0.8413 Precision: 0.8494 Recall: 0.8414 F1: 0.8417

Epoch 1/29
----------
train Loss: 0.5360 Acc: 0.8162 Precision: 0.8152 Recall: 0.8147 F1: 0.8145
val Loss: 0.3300 Acc: 0.8871 Precision: 0.8956 Recall: 0.8939 F1: 0.8886

Epoch 2/29
----------
train Loss: 0.4119 Acc: 0.8601 Precision: 0.8597 Recall: 0.8586 F1: 0.8590
val Loss: 0.2110 Acc: 0.9198 Precision: 0.9191 Recall: 0.9188 F1: 0.9175

Epoch 3/29
----------
train Loss: 0.3296 Acc: 0.8897 Precision: 0.8899 Recall: 0.8897 F1: 0.8896
val Loss: 0.2328 Acc: 0.9193 Precision: 0.9214 Recall: 0.9225 F1: 0.9184

Epoch 4/29
----------
train Loss: 0.3005 Acc: 0.8981 Precision: 0.8994 Recall: 0.8987 F1: 0.8989
val Loss: 0.1753 Acc: 0.9435 Precision: 0.9437 Recall: 0.9437 F1: 0.9428

Epoch 5/29
----------
train Loss: 0.2586 Acc: 0.9131 Precision: 0.9133 Recall: 0.9126 F1: 0.9129
val Loss: 0

d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



resnet50 Classifier:
Linear(in_features=2048, out_features=9, bias=True)
resnet50 Parameters: 89.75 MB
Epoch 0/29
----------
train Loss: 1.4992 Acc: 0.4642 Precision: 0.4462 Recall: 0.4607 F1: 0.4459
val Loss: 1.0697 Acc: 0.6273 Precision: 0.6626 Recall: 0.6198 F1: 0.5963

Epoch 1/29
----------
train Loss: 1.0947 Acc: 0.6173 Precision: 0.6111 Recall: 0.6136 F1: 0.6059
val Loss: 0.7891 Acc: 0.7357 Precision: 0.7433 Recall: 0.7397 F1: 0.7366

Epoch 2/29
----------
train Loss: 0.9745 Acc: 0.6561 Precision: 0.6496 Recall: 0.6518 F1: 0.6477
val Loss: 0.9443 Acc: 0.6595 Precision: 0.6835 Recall: 0.6636 F1: 0.6579

Epoch 3/29
----------
train Loss: 0.8988 Acc: 0.6784 Precision: 0.6724 Recall: 0.6732 F1: 0.6701
val Loss: 0.6926 Acc: 0.7623 Precision: 0.7683 Recall: 0.7618 F1: 0.7596

Epoch 4/29
----------
train Loss: 0.7921 Acc: 0.7214 Precision: 0.7187 Recall: 0.7169 F1: 0.7157
val Loss: 0.9927 Acc: 0.6827 Precision: 0.7748 Recall: 0.6747 F1: 0.6584

Epoch 5/29
----------
train Loss: 0.7372 

d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


train Loss: 1.1437 Acc: 0.6117 Precision: 0.6114 Recall: 0.5975 F1: 0.5831
val Loss: 0.6938 Acc: 0.7595 Precision: 0.7871 Recall: 0.7529 F1: 0.7494

Epoch 1/29
----------
train Loss: 0.6870 Acc: 0.7574 Precision: 0.7503 Recall: 0.7519 F1: 0.7499
val Loss: 0.3995 Acc: 0.8498 Precision: 0.8497 Recall: 0.8429 F1: 0.8427

Epoch 2/29
----------
train Loss: 0.5446 Acc: 0.8087 Precision: 0.8081 Recall: 0.8056 F1: 0.8061
val Loss: 0.3277 Acc: 0.8826 Precision: 0.8879 Recall: 0.8757 F1: 0.8783

Epoch 3/29
----------
train Loss: 0.4734 Acc: 0.8364 Precision: 0.8359 Recall: 0.8349 F1: 0.8350
val Loss: 0.2961 Acc: 0.9063 Precision: 0.9059 Recall: 0.9069 F1: 0.9054

Epoch 4/29
----------
train Loss: 0.4002 Acc: 0.8644 Precision: 0.8638 Recall: 0.8627 F1: 0.8630
val Loss: 0.2863 Acc: 0.8950 Precision: 0.8958 Recall: 0.8948 F1: 0.8941

Epoch 5/29
----------
train Loss: 0.3569 Acc: 0.8780 Precision: 0.8766 Recall: 0.8760 F1: 0.8761
val Loss: 0.2881 Acc: 0.9051 Precision: 0.9082 Recall: 0.9049 F1: 0.90

d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


train Loss: 0.9253 Acc: 0.6716 Precision: 0.6663 Recall: 0.6681 F1: 0.6653
val Loss: 0.7242 Acc: 0.7708 Precision: 0.7867 Recall: 0.7763 F1: 0.7767

Epoch 1/29
----------
train Loss: 0.6078 Acc: 0.7925 Precision: 0.7909 Recall: 0.7902 F1: 0.7901
val Loss: 0.4118 Acc: 0.8532 Precision: 0.8581 Recall: 0.8585 F1: 0.8530

Epoch 2/29
----------
train Loss: 0.4731 Acc: 0.8409 Precision: 0.8407 Recall: 0.8400 F1: 0.8402
val Loss: 0.2722 Acc: 0.9068 Precision: 0.9098 Recall: 0.9026 F1: 0.9040

Epoch 3/29
----------
train Loss: 0.3682 Acc: 0.8734 Precision: 0.8740 Recall: 0.8725 F1: 0.8731
val Loss: 0.3018 Acc: 0.9119 Precision: 0.9237 Recall: 0.9079 F1: 0.9115

Epoch 4/29
----------
train Loss: 0.3176 Acc: 0.8939 Precision: 0.8934 Recall: 0.8933 F1: 0.8933
val Loss: 0.2964 Acc: 0.9001 Precision: 0.9063 Recall: 0.8953 F1: 0.8973

Epoch 5/29
----------
train Loss: 0.2858 Acc: 0.9025 Precision: 0.9029 Recall: 0.9023 F1: 0.9025
val Loss: 0.2770 Acc: 0.9006 Precision: 0.9083 Recall: 0.9042 F1: 0.90

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms
from torchvision.models.efficientnet import MBConv
import time
import copy
import shutil
from tqdm import tqdm

# 设置随机种子确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 配置参数
DATA_DIR = r"D:\homework\论文\论文\project\dataset\archive\Rice_Leaf_AUG\Rice_Leaf_AUG"  # 原始数据集路径
OUTPUT_DIR = "dataset_split"  # 划分后的数据集保存路径
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
NUM_WORKERS = 4  # 数据加载的线程数
SPLIT_RATIO = [0.7, 0.15, 0.15]  # 训练集、验证集、测试集比例

# 模型配置
MODELS = {
    "efficientnet-b0": {
        "model": models.efficientnet_b0,
        "pretrained": True,
        "feature_size": 1280  # EfficientNet-B0的特征维度
    },
}

# 检查GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 注意力模块定义
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction_ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channels // reduction_ratio, channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return x * self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(out)
        return x * self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channels, reduction_ratio)
        self.spatial_attention = SpatialAttention()
    
    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

def add_cbam_to_efficientnet(model):
    """为EfficientNet添加CBAM模块"""
    features = []
    for layer in model.features.children():
        features.append(layer)
        if isinstance(layer, MBConv):
            # 获取当前MBConv层的输出通道数
            out_channels = layer.out_channels
            # 添加CBAM模块
            features.append(CBAM(out_channels))
    # 重构features模块
    model.features = nn.Sequential(*features)
    return model

def split_dataset(data_dir, output_dir, split_ratio):
    """将数据集划分为训练集、验证集和测试集"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    subdirs = [f.name for f in os.scandir(data_dir) if f.is_dir()]
    
    for subdir in subdirs:
        print(f"Processing class: {subdir}")
        class_dir = os.path.join(data_dir, subdir)
        files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # 划分数据集
        train_files, test_files = train_test_split(files, test_size=split_ratio[2], random_state=42)
        train_files, val_files = train_test_split(train_files, test_size=split_ratio[1]/(split_ratio[0]+split_ratio[1]), random_state=42)
        
        # 创建输出目录
        for split in ["train", "val", "test"]:
            split_dir = os.path.join(output_dir, split, subdir)
            os.makedirs(split_dir, exist_ok=True)
        
        # 复制文件
        def copy_files(files_list, split_name):
            for file_name in tqdm(files_list, desc=f"Copying {split_name} files"):
                src = os.path.join(class_dir, file_name)
                dst = os.path.join(output_dir, split_name, subdir, file_name)
                shutil.copy(src, dst)
        
        copy_files(train_files, "train")
        copy_files(val_files, "val")
        copy_files(test_files, "test")
    
    print(f"Dataset split completed. Saved to {output_dir}")
    return output_dir

def create_data_transforms():
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(p=0.1),
            transforms.RandomRotation(30),
            transforms.ColorJitter(
                brightness=0.3,
                contrast=0.3,
                saturation=0.3,
                hue=0.1
            ),
            transforms.RandomGrayscale(p=0.1),
            transforms.RandomAffine(
                degrees=0,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1),
                shear=10
            ),
            transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def create_data_loaders(data_dir):
    """创建数据加载器"""
    data_transforms = create_data_transforms()
    
    # 创建数据集
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'val', 'test']}
    
    # 创建数据加载器
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                 shuffle=True if x == 'train' else False,
                                 num_workers=NUM_WORKERS)
                   for x in ['train', 'val', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes
    
    return dataloaders, dataset_sizes, class_names

def initialize_model(model_name, num_classes, feature_extract=True):
    """初始化预训练模型"""
    model_info = MODELS[model_name]
    model_ft = model_info["model"](pretrained=model_info["pretrained"])
    
    # 添加注意力模块（仅对efficientnet）
    if model_name == "efficientnet-b0":
        model_ft = add_cbam_to_efficientnet(model_ft)
    
    # 冻结预训练模型的参数
    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False
    
    # 修改分类器
    if model_name == "efficientnet-b0":
        in_features = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(in_features, num_classes)
    
    return model_ft

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, model_name="model"):
    """训练模型"""
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # 记录训练历史
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            # 迭代数据
            progress_bar = tqdm(enumerate(dataloaders[phase]), total=len(dataloaders[phase]))
            for i, (inputs, labels) in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 零梯度
                optimizer.zero_grad()
                
                # 前向传播
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # 反向传播+优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                # 更新进度条
                progress_bar.set_description(f"{phase} Loss: {running_loss/(i*BATCH_SIZE+inputs.size(0)):.4f} Acc: {running_corrects/(i*BATCH_SIZE+inputs.size(0)):.4f}")
            
            if phase == 'train' and scheduler:
                scheduler.step()
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # 记录历史
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            
            # 保存最佳模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), f"{model_name}_best_model.pth")
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model, history

def evaluate_model(model, dataloader, class_names, model_name):
    """评估模型"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # 打印分类报告+
    print(f"\n{classification_report(all_labels, all_preds, target_names=class_names)}")
    
    # 计算混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    plot_confusion_matrix(cm, class_names, f"{model_name}_confusion_matrix.png")
    
    return all_preds, all_labels

def plot_confusion_matrix(cm, classes, save_path, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """绘制混淆矩阵"""
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(save_path)
    plt.close()

def plot_training_curves(histories, save_path='training_curves.png'):
    """绘制训练曲线"""
    plt.figure(figsize=(12, 10))
    
    # 绘制准确率曲线
    plt.subplot(2, 1, 1)
    for model_name, history in histories.items():
        plt.plot(history['train_acc'], label=f'{model_name} Train')
        plt.plot(history['val_acc'], label=f'{model_name} Val')
    
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    # 绘制损失曲线
    plt.subplot(2, 1, 2)
    for model_name, history in histories.items():
        plt.plot(history['train_loss'], label=f'{model_name} Train')
        plt.plot(history['val_loss'], label=f'{model_name} Val')
    
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def main():
    """主函数"""
    # 划分数据集
    split_dir = split_dataset(DATA_DIR, OUTPUT_DIR, SPLIT_RATIO)
    
    # 创建数据加载器
    dataloaders, dataset_sizes, class_names = create_data_loaders(split_dir)
    print(f"Classes: {class_names}")
    num_classes = len(class_names)
    
    all_histories = {}
    
    # 训练所有模型
    for model_name in MODELS.keys():
        print(f"\n===== Training {model_name} model =====")
        
        # 初始化模型
        model_ft = initialize_model(model_name, num_classes, feature_extract=False)
        model_ft = model_ft.to(device)
        
        # 设置损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
        
        # 训练模型
        model_ft, history = train_model(
            model_ft, dataloaders, criterion, optimizer_ft, exp_lr_scheduler,
            num_epochs=EPOCHS, model_name=model_name
        )
        
        # 保存最终模型
        torch.save(model_ft.state_dict(), f"{model_name}_final_model.pth")
        
        # 评估模型
        print(f"\nEvaluating {model_name} model:")
        evaluate_model(model_ft, dataloaders['test'], class_names, model_name)
        
        # 保存训练历史
        all_histories[model_name] = history
    
    # 绘制训练曲线
    plot_training_curves(all_histories, save_path='all_models_training_curves.png')
    
    print("\n===== Training completed for all models =====")

if __name__ == "__main__":
    main()

d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0
GPU: NVIDIA GeForce RTX 3060 Ti
Processing class: Bacterial Leaf Blight


Copying test files: 100%|██████████| 180/180 [00:01<00:00, 154.15it/s]


Processing class: Brown Spot


Copying test files: 100%|██████████| 232/232 [00:01<00:00, 161.76it/s]


...

train Loss: 0.0400 Acc: 0.9863


val Loss: 0.0092 Acc: 0.9977: 100%|██████████| 56/56 [00:08<00:00,  6.86it/s]


val Loss: 0.0092 Acc: 0.9977

Training complete in 42m 13s
Best val Acc: 0.998306

Evaluating efficientnet-b0 model:


Evaluating: 100%|██████████| 56/56 [00:20<00:00,  2.79it/s]



                        precision    recall  f1-score   support

 Bacterial Leaf Blight       0.99      0.98      0.99       180
            Brown Spot       1.00      0.99      1.00       232
     Healthy Rice Leaf       1.00      1.00      1.00       163
            Leaf Blast       0.98      1.00      0.99       263
            Leaf scald       1.00      0.98      0.99       200
Narrow Brown Leaf Spot       0.99      1.00      0.99       144
            Neck Blast       1.00      1.00      1.00       150
            Rice Hispa       1.00      1.00      1.00       195
         Sheath Blight       1.00      1.00      1.00       245

              accuracy                           0.99      1772
             macro avg       0.99      0.99      0.99      1772
          weighted avg       0.99      0.99      0.99      1772

Confusion matrix, without normalization

===== Training completed for all models =====


: 

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report, confusion_matrix
import time
import copy

# 设置随机种子确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 配置参数
DATA_DIR = r"D:\homework\论文\论文\project\code\训练\dataset_split"
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 1
NUM_WORKERS = 4
FREEZE_LAYERS = True  # 是否冻结部分层

# 模型配置
MODELS = {
        "efficientnet-b0": {
        "model": models.efficientnet_b0,
        "pretrained": True,
        "feature_size": 1280,
        "classifier": lambda feat_size, num_classes: nn.Linear(feat_size, num_classes),
        "freeze_func": lambda model: [
            # 冻结特征提取器的前半部分
            param.requires_grad_(False) for param in list(model.features.parameters())[:len(list(model.features.parameters()))//2]
        ]
    },
    "resnet50": {
        "model": models.resnet50,
        "pretrained": True,
        "feature_size": 2048,
        "classifier": lambda feat_size, num_classes: nn.Linear(feat_size, num_classes),
        "freeze_func": lambda model: [
            # 冻结前3个残差块（共4个）
            param.requires_grad_(False) for param in list(model.parameters())[:len(list(model.parameters()))//2]
        ]
    },
    "shufflenet_v2": {
        "model": models.shufflenet_v2_x1_0,
        "pretrained": True,
        "feature_size": 1024,
        "classifier": lambda feat_size, num_classes: nn.Linear(feat_size, num_classes),
        "freeze_func": lambda model: [
            # 冻结前半部分层
            param.requires_grad_(False) for param in list(model.parameters())[:len(list(model.parameters()))//2]
        ]
    },

    "mobilenet_v3": {
        "model": models.mobilenet_v3_large,
        "pretrained": True,
        "feature_size": 960,
        "classifier": lambda feat_size, num_classes: nn.Sequential(
            nn.Linear(feat_size, 1280),
            nn.Hardswish(),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1280, num_classes)
        ),
        "freeze_func": lambda model: [
            # 冻结特征提取器的前半部分
            param.requires_grad_(False) for param in list(model.features.parameters())[:len(list(model.features.parameters()))//2]
        ]
    }
}

# 检查GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def create_data_transforms():
    """创建数据转换"""
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(IMAGE_SIZE),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def create_data_loaders(data_dir):
    """创建数据加载器"""
    data_transforms = create_data_transforms()
    
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                      for x in ['train', 'val', 'test']}
    
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                 shuffle=True if x == 'train' else False,
                                 num_workers=NUM_WORKERS)
                   for x in ['train', 'val', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes
    
    return dataloaders, dataset_sizes, class_names

def initialize_model(model_name, num_classes):
    """初始化预训练模型并冻结部分层"""
    model_info = MODELS[model_name]
    model = model_info["model"](pretrained=model_info["pretrained"])
    
    # 修改分类器
    if model_name == "resnet50":
        model.fc = model_info["classifier"](model_info["feature_size"], num_classes)
    elif model_name == "shufflenet_v2":
        model.fc = model_info["classifier"](model_info["feature_size"], num_classes)
    elif model_name == "efficientnet-b0":
        model.classifier[1] = model_info["classifier"](model_info["feature_size"], num_classes)
    elif model_name == "mobilenet_v3":
        model.classifier = model_info["classifier"](model_info["feature_size"], num_classes)
    
    # 冻结部分层
    if FREEZE_LAYERS:
        print(f"Freezing layers for {model_name}...")
        model_info["freeze_func"](model)
        
        # 计算可训练参数
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Trainable params: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
    
    return model.to(device)

def count_parameters(model):
    """计算模型参数量（MB）"""
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return params * 4 / (1024 * 1024)  # 转换为MB

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, model_name="model"):
    """训练模型并记录指标"""
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # 记录训练历史
    history = {
        'train_loss': [], 'train_acc': [], 'train_precision': [], 'train_recall': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_precision': [], 'val_recall': [], 'val_f1': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 训练模式
            else:
                model.eval()   # 评估模式
            
            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []
            
            # 迭代数据
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 零梯度
                optimizer.zero_grad()
                
                # 前向传播
                # 只有在训练时才跟踪历史
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # 只有在训练阶段才进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            # 计算epoch指标
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            # 计算分类报告
            report = classification_report(all_labels, all_preds, output_dict=True, zero_division=0)
            macro_precision = report['macro avg']['precision']
            macro_recall = report['macro avg']['recall']
            macro_f1 = report['macro avg']['f1-score']
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} '
                  f'Precision: {macro_precision:.4f} Recall: {macro_recall:.4f} F1: {macro_f1:.4f}')
            
            # 记录历史
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            history[f'{phase}_precision'].append(macro_precision)
            history[f'{phase}_recall'].append(macro_recall)
            history[f'{phase}_f1'].append(macro_f1)
            
            # 深拷贝模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), f"{model_name}_best_model.pth")
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model, history

def evaluate_model(model, dataloader, class_names, model_name):
    """评估模型"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # 打印分类报告
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    print(f"\n{model_name} Test Metrics:")
    print(f"Accuracy: {report['accuracy']:.4f}")
    print(f"Macro Precision: {report['macro avg']['precision']:.4f}")
    print(f"Macro Recall: {report['macro avg']['recall']:.4f}")
    print(f"Macro F1-Score: {report['macro avg']['f1-score']:.4f}")
    
    # 计算混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    
    return {
        'accuracy': report['accuracy'],
        'precision': report['macro avg']['precision'],
        'recall': report['macro avg']['recall'],
        'f1': report['macro avg']['f1-score'],
        'confusion_matrix': cm
    }

def plot_training_curves(histories, metrics, save_path):
    """绘制训练曲线"""
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metrics):
        plt.subplot(2, 3, i+1)
        
        for model_name, history in histories.items():
            plt.plot(history[f'train_{metric}'], label=f'{model_name} Train')
            plt.plot(history[f'val_{metric}'], label=f'{model_name} Val')
        
        plt.title(f'Model {metric.capitalize()}')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def compare_models_table(model_metrics, model_sizes):
    """生成模型对比表格"""
    print("\n===== Model Comparison =====")
    print(f"{'Model':<15} {'Accuracy(%)':<12} {'Precision(%)':<12} {'Recall(%)':<12} {'F1-Score(%)':<12} {'Params(MB)':<12}")
    print("-" * 75)
    
    for model_name, metrics in model_metrics.items():
        print(f"{model_name:<15} {metrics['accuracy']*100:<12.2f} {metrics['precision']*100:<12.2f} "
              f"{metrics['recall']*100:<12.2f} {metrics['f1']*100:<12.2f} {model_sizes[model_name]:<12.2f}")

def main():
    """主函数"""
    # 创建数据加载器
    dataloaders, dataset_sizes, class_names = create_data_loaders(DATA_DIR)
    print(f"Classes: {class_names}")
    num_classes = len(class_names)
    
    all_histories = {}
    all_metrics = {}
    model_sizes = {}
    
    # 训练所有模型
    for model_name in MODELS.keys():
        print(f"\n===== Training {model_name} model =====")
        
        # 初始化模型
        model = initialize_model(model_name, num_classes)
        model = model.to(device)
        
        # 计算模型大小
        model_size = count_parameters(model)
        model_sizes[model_name] = model_size
        print(f"{model_name} Trainable Parameters: {model_size:.2f} MB")
        
        # 设置损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)
        
        # 训练模型
        model, history = train_model(
            model, dataloaders, criterion, optimizer,
            num_epochs=EPOCHS, model_name=model_name
        )
        
        # 评估模型
        print(f"\nEvaluating {model_name} model:")
        metrics = evaluate_model(model, dataloaders['test'], class_names, model_name)
        all_metrics[model_name] = metrics
        
        # 保存训练历史
        all_histories[model_name] = history
    
    # 绘制训练曲线
    plot_training_curves(
        all_histories, 
        metrics=['loss', 'acc', 'precision', 'recall', 'f1'],
        save_path='training_curves_comparison.png'
    )
    
    # 生成模型对比表格
    compare_models_table(all_metrics, model_sizes)
    
    print("\n===== Training and evaluation completed for all models =====")
    print("Training curves saved to 'training_curves_comparison.png'")

if __name__ == "__main__":
    main()

Using device: cuda:0
Classes: ['Bacterial Leaf Blight', 'Brown Spot', 'Healthy Rice Leaf', 'Leaf Blast', 'Leaf scald', 'Narrow Brown Leaf Spot', 'Neck Blast', 'Rice Hispa', 'Sheath Blight']

===== Training efficientnet-b0 model =====
Freezing layers for efficientnet-b0...
Trainable params: 3,672,017 (91.36%)
efficientnet-b0 Trainable Parameters: 14.01 MB
Epoch 0/0
----------


KeyboardInterrupt: 

最终优化 冻结层 CA注意力机制

In [ ]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.optim import lr_scheduler
# from torch.utils.data import DataLoader, Subset
# from torchvision import datasets, models, transforms
# from torchvision.models.efficientnet import MBConv
# import time
# import copy
# import shutil
# from tqdm import tqdm

# # 设置随机种子确保结果可复现
# torch.manual_seed(42)
# np.random.seed(42)

# # 配置参数
# DATA_DIR = r"D:\homework\论文\论文\project\dataset\archive\Rice_Leaf_AUG\Rice_Leaf_AUG"  # 原始数据集路径
# OUTPUT_DIR = "dataset_split"  # 划分后的数据集保存路径
# IMAGE_SIZE = 224
# BATCH_SIZE = 32
# EPOCHS = 50
# NUM_WORKERS = 4  # 数据加载的线程数
# SPLIT_RATIO = [0.7, 0.15, 0.15]  # 训练集、验证集、测试集比例

# # 模型配置
# MODELS = {
#     "efficientnet-b0": {
#         "model": models.efficientnet_b0,
#         "pretrained": True,
#         "feature_size": 1280  # EfficientNet-B0的特征维度
#     },
# }

# # 检查GPU是否可用
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
# if device.type == 'cuda':
#     print(f"GPU: {torch.cuda.get_device_name(0)}")

# # 注意力模块定义
# class ChannelAttention(nn.Module):
#     def __init__(self, channels, reduction_ratio=16):
#         super(ChannelAttention, self).__init__()
#         self.avg_pool = nn.AdaptiveAvgPool2d(1)
#         self.max_pool = nn.AdaptiveMaxPool2d(1)
#         self.fc = nn.Sequential(
#             nn.Conv2d(channels, channels // reduction_ratio, 1, bias=False),
#             nn.ReLU(),
#             nn.Conv2d(channels // reduction_ratio, channels, 1, bias=False)
#         )
#         self.sigmoid = nn.Sigmoid()
    
#     def forward(self, x):
#         avg_out = self.fc(self.avg_pool(x))
#         max_out = self.fc(self.max_pool(x))
#         out = avg_out + max_out
#         return x * self.sigmoid(out)

# class SpatialAttention(nn.Module):
#     def __init__(self, kernel_size=7):
#         super(SpatialAttention, self).__init__()
#         self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
#         self.sigmoid = nn.Sigmoid()
    
#     def forward(self, x):
#         avg_out = torch.mean(x, dim=1, keepdim=True)
#         max_out, _ = torch.max(x, dim=1, keepdim=True)
#         out = torch.cat([avg_out, max_out], dim=1)
#         out = self.conv(out)
#         return x * self.sigmoid(out)

# class CBAM(nn.Module):
#     def __init__(self, channels, reduction_ratio=16):
#         super(CBAM, self).__init__()
#         self.channel_attention = ChannelAttention(channels, reduction_ratio)
#         self.spatial_attention = SpatialAttention()
    
#     def forward(self, x):
#         x = self.channel_attention(x)
#         x = self.spatial_attention(x)
#         return x

# def add_cbam_to_efficientnet(model):
#     """为EfficientNet添加CBAM模块"""
#     features = []
#     for layer in model.features.children():
#         features.append(layer)
#         if isinstance(layer, MBConv):
#             # 获取当前MBConv层的输出通道数
#             out_channels = layer.out_channels
#             # 添加CBAM模块
#             features.append(CBAM(out_channels))
#     # 重构features模块
#     model.features = nn.Sequential(*features)
#     return model

# def split_dataset(data_dir, output_dir, split_ratio):
#     """将数据集划分为训练集、验证集和测试集"""
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     subdirs = [f.name for f in os.scandir(data_dir) if f.is_dir()]
    
#     for subdir in subdirs:
#         print(f"Processing class: {subdir}")
#         class_dir = os.path.join(data_dir, subdir)
#         files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
#         # 划分数据集
#         train_files, test_files = train_test_split(files, test_size=split_ratio[2], random_state=42)
#         train_files, val_files = train_test_split(train_files, test_size=split_ratio[1]/(split_ratio[0]+split_ratio[1]), random_state=42)
        
#         # 创建输出目录
#         for split in ["train", "val", "test"]:
#             split_dir = os.path.join(output_dir, split, subdir)
#             os.makedirs(split_dir, exist_ok=True)
        
#         # 复制文件
#         def copy_files(files_list, split_name):
#             for file_name in tqdm(files_list, desc=f"Copying {split_name} files"):
#                 src = os.path.join(class_dir, file_name)
#                 dst = os.path.join(output_dir, split_name, subdir, file_name)
#                 shutil.copy(src, dst)
        
#         copy_files(train_files, "train")
#         copy_files(val_files, "val")
#         copy_files(test_files, "test")
    
#     print(f"Dataset split completed. Saved to {output_dir}")
#     return output_dir

# def create_data_transforms():
#     data_transforms = {
#         'train': transforms.Compose([
#             transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomVerticalFlip(p=0.1),
#             transforms.RandomRotation(30),
#             transforms.ColorJitter(
#                 brightness=0.3,
#                 contrast=0.3,
#                 saturation=0.3,
#                 hue=0.1
#             ),
#             transforms.RandomGrayscale(p=0.1),
#             transforms.RandomAffine(
#                 degrees=0,
#                 translate=(0.1, 0.1),
#                 scale=(0.9, 1.1),
#                 shear=10
#             ),
#             transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
#             transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
#             transforms.ToTensor(),
#             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#         ]),
#         'val': transforms.Compose([
#             transforms.Resize(IMAGE_SIZE),
#             transforms.CenterCrop(IMAGE_SIZE),
#             transforms.ToTensor(),
#             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#         ]),
#         'test': transforms.Compose([
#             transforms.Resize(IMAGE_SIZE),
#             transforms.CenterCrop(IMAGE_SIZE),
#             transforms.ToTensor(),
#             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#         ]),
#     }
#     return data_transforms

# def create_data_loaders(data_dir):
#     """创建数据加载器"""
#     data_transforms = create_data_transforms()
    
#     # 创建数据集
#     image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                               data_transforms[x])
#                       for x in ['train', 'val', 'test']}
    
#     # 创建数据加载器
#     dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
#                                  shuffle=True if x == 'train' else False,
#                                  num_workers=NUM_WORKERS)
#                    for x in ['train', 'val', 'test']}
    
#     dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
#     class_names = image_datasets['train'].classes
    
#     return dataloaders, dataset_sizes, class_names

# def initialize_model(model_name, num_classes, feature_extract=True):
#     """初始化预训练模型"""
#     model_info = MODELS[model_name]
#     model_ft = model_info["model"](pretrained=model_info["pretrained"])
    
#     # 添加注意力模块（仅对efficientnet）
#     if model_name == "efficientnet-b0":
#         model_ft = add_cbam_to_efficientnet(model_ft)
    
#     # 冻结预训练模型的参数
#     if feature_extract:
#         for param in model_ft.parameters():
#             param.requires_grad = False
    
#     # 修改分类器
#     if model_name == "efficientnet-b0":
#         in_features = model_ft.classifier[1].in_features
#         model_ft.classifier[1] = nn.Linear(in_features, num_classes)
    
#     return model_ft

# def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, model_name="model"):
#     """训练模型"""
#     since = time.time()
    
#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0
    
#     # 记录训练历史
#     history = {
#         'train_loss': [],
#         'train_acc': [],
#         'val_loss': [],
#         'val_acc': []
#     }
    
#     for epoch in range(num_epochs):
#         print(f'Epoch {epoch}/{num_epochs-1}')
#         print('-' * 10)
        
#         # 每个epoch都有一个训练和验证阶段
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()
#             else:
#                 model.eval()
            
#             running_loss = 0.0
#             running_corrects = 0
            
#             # 迭代数据
#             progress_bar = tqdm(enumerate(dataloaders[phase]), total=len(dataloaders[phase]))
#             for i, (inputs, labels) in progress_bar:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)
                
#                 # 零梯度
#                 optimizer.zero_grad()
                
#                 # 前向传播
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)
                    
#                     # 反向传播+优化
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()
                
#                 # 统计
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)
                
#                 # 更新进度条
#                 progress_bar.set_description(f"{phase} Loss: {running_loss/(i*BATCH_SIZE+inputs.size(0)):.4f} Acc: {running_corrects/(i*BATCH_SIZE+inputs.size(0)):.4f}")
            
#             if phase == 'train' and scheduler:
#                 scheduler.step()
            
#             epoch_loss = running_loss / len(dataloaders[phase].dataset)
#             epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
#             print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
#             # 记录历史
#             history[f'{phase}_loss'].append(epoch_loss)
#             history[f'{phase}_acc'].append(epoch_acc.item())
            
#             # 保存最佳模型
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())
#                 torch.save(model.state_dict(), f"{model_name}_best_model.pth")
        
#         print()
    
#     time_elapsed = time.time() - since
#     print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
#     print(f'Best val Acc: {best_acc:4f}')
    
#     # 加载最佳模型权重
#     model.load_state_dict(best_model_wts)
#     return model, history

# def evaluate_model(model, dataloader, class_names, model_name):
#     """评估模型"""
#     model.eval()
#     all_preds = []
#     all_labels = []
    
#     with torch.no_grad():
#         for inputs, labels in tqdm(dataloader, desc="Evaluating"):
#             inputs = inputs.to(device)
#             labels = labels.to(device)
            
#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
            
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
    
#     # 打印分类报告+
#     print(f"\n{classification_report(all_labels, all_preds, target_names=class_names)}")
    
#     # 计算混淆矩阵
#     cm = confusion_matrix(all_labels, all_preds)
#     plot_confusion_matrix(cm, class_names, f"{model_name}_confusion_matrix.png")
    
#     return all_preds, all_labels

# def plot_confusion_matrix(cm, classes, save_path, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
#     """绘制混淆矩阵"""
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')
    
#     plt.figure(figsize=(10, 10))
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)
    
#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i in range(cm.shape[0]):
#         for j in range(cm.shape[1]):
#             plt.text(j, i, format(cm[i, j], fmt),
#                      horizontalalignment="center",
#                      color="white" if cm[i, j] > thresh else "black")
    
#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.savefig(save_path)
#     plt.close()

# def plot_training_curves(histories, save_path='training_curves.png'):
#     """绘制训练曲线"""
#     plt.figure(figsize=(12, 10))
    
#     # 绘制准确率曲线
#     plt.subplot(2, 1, 1)
#     for model_name, history in histories.items():
#         plt.plot(history['train_acc'], label=f'{model_name} Train')
#         plt.plot(history['val_acc'], label=f'{model_name} Val')
    
#     plt.title('Model Accuracy')
#     plt.ylabel('Accuracy')
#     plt.xlabel('Epoch')
#     plt.legend(loc='lower right')
#     plt.grid(True)
    
#     # 绘制损失曲线
#     plt.subplot(2, 1, 2)
#     for model_name, history in histories.items():
#         plt.plot(history['train_loss'], label=f'{model_name} Train')
#         plt.plot(history['val_loss'], label=f'{model_name} Val')
    
#     plt.title('Model Loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(loc='upper right')
#     plt.grid(True)
    
#     plt.tight_layout()
#     plt.savefig(save_path)
#     plt.close()

# def main():
#     """主函数"""
#     # 划分数据集
#     split_dir = split_dataset(DATA_DIR, OUTPUT_DIR, SPLIT_RATIO)
    
#     # 创建数据加载器
#     dataloaders, dataset_sizes, class_names = create_data_loaders(split_dir)
#     print(f"Classes: {class_names}")
#     num_classes = len(class_names)
    
#     all_histories = {}
    
#     # 训练所有模型
#     for model_name in MODELS.keys():
#         print(f"\n===== Training {model_name} model =====")
        
#         # 初始化模型
#         model_ft = initialize_model(model_name, num_classes, feature_extract=False)
#         model_ft = model_ft.to(device)
        
#         # 设置损失函数和优化器
#         criterion = nn.CrossEntropyLoss()
#         optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
#         exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
        
#         # 训练模型
#         model_ft, history = train_model(
#             model_ft, dataloaders, criterion, optimizer_ft, exp_lr_scheduler,
#             num_epochs=EPOCHS, model_name=model_name
#         )
        
#         # 保存最终模型
#         torch.save(model_ft.state_dict(), f"{model_name}_final_model.pth")
        
#         # 评估模型
#         print(f"\nEvaluating {model_name} model:")
#         evaluate_model(model_ft, dataloaders['test'], class_names, model_name)
        
#         # 保存训练历史
#         all_histories[model_name] = history
    
#     # 绘制训练曲线
#     plot_training_curves(all_histories, save_path='all_models_training_curves.png')
    
#     print("\n===== Training completed for all models =====")

# if __name__ == "__main__":
#     main()

d:\config\anaconda3\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0
GPU: NVIDIA GeForce RTX 3060 Ti
Processing class: Bacterial Leaf Blight


Copying test files: 100%|██████████| 180/180 [00:00<00:00, 1173.72it/s]


Processing class: Brown Spot


Copying test files: 100%|██████████| 232/232 [00:00<00:00, 1216.81it/s]


...

train Loss: 0.0406 Acc: 0.9880


val Loss: 0.0164 Acc: 0.9938: 100%|██████████| 56/56 [00:08<00:00,  6.83it/s]


val Loss: 0.0164 Acc: 0.9938

Training complete in 70m 48s
Best val Acc: 0.994353

Evaluating efficientnet-b0 model:


Evaluating: 100%|██████████| 56/56 [00:20<00:00,  2.70it/s]



                        precision    recall  f1-score   support

 Bacterial Leaf Blight       0.99      0.98      0.99       180
            Brown Spot       1.00      1.00      1.00       232
     Healthy Rice Leaf       0.99      1.00      0.99       163
            Leaf Blast       0.98      1.00      0.99       263
            Leaf scald       1.00      0.98      0.99       200
Narrow Brown Leaf Spot       1.00      0.99      0.99       144
            Neck Blast       1.00      1.00      1.00       150
            Rice Hispa       1.00      1.00      1.00       195
         Sheath Blight       0.99      1.00      0.99       245

              accuracy                           0.99      1772
             macro avg       0.99      0.99      0.99      1772
          weighted avg       0.99      0.99      0.99      1772

Confusion matrix, without normalization

===== Training completed for all models =====


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms
import time
import copy
import shutil
from tqdm import tqdm

# 设置随机种子确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 配置参数
DATA_DIR = r"D:\homework\论文\论文\project\dataset\archive\Rice_Leaf_AUG\Rice_Leaf_AUG"  # 原始数据集路径
OUTPUT_DIR = "dataset_split"  # 划分后的数据集保存路径
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 1
NUM_WORKERS = 4  # 数据加载的线程数
SPLIT_RATIO = [0.7, 0.15, 0.15]  # 训练集、验证集、测试集比例

# 模型配置
MODELS = {
    #     "resnet50": {
    #     "model": models.resnet50,
    #     "pretrained": True,
    #     "feature_size": 2048  # ResNet50的特征维度
    # },
    # "vgg16": {
    #     "model": models.vgg16,
    #     "pretrained": True,
    #     "feature_size": 512 * 7 * 7  # VGG16的特征维度
    # },
    # "efficientnet-b0": {
    #     "model": models.efficientnet_b0,
    #     "pretrained": True,
    #     "feature_size": 1280  # EfficientNet-B0的特征维度
    # },
    "mobilenet_v3": {
        "model": models.mobilenet_v3_large,
        "pretrained": True,
        "feature_size": 960  # MobileNetV3的特征维度
    },

}

# 检查GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

def split_dataset(data_dir, output_dir, split_ratio):
    """将数据集划分为训练集、验证集和测试集"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    subdirs = [f.name for f in os.scandir(data_dir) if f.is_dir()]
    
    for subdir in subdirs:
        print(f"Processing class: {subdir}")
        class_dir = os.path.join(data_dir, subdir)
        files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # 划分数据集
        train_files, test_files = train_test_split(files, test_size=split_ratio[2], random_state=42)
        train_files, val_files = train_test_split(train_files, test_size=split_ratio[1]/(split_ratio[0]+split_ratio[1]), random_state=42)
        
        # 创建输出目录
        for split in ["train", "val", "test"]:
            split_dir = os.path.join(output_dir, split, subdir)
            os.makedirs(split_dir, exist_ok=True)
        
        # 复制文件
        def copy_files(files_list, split_name):
            for file_name in tqdm(files_list, desc=f"Copying {split_name} files"):
                src = os.path.join(class_dir, file_name)
                dst = os.path.join(output_dir, split_name, subdir, file_name)
                shutil.copy(src, dst)
        
        copy_files(train_files, "train")
        copy_files(val_files, "val")
        copy_files(test_files, "test")
    
    print(f"Dataset split completed. Saved to {output_dir}")
    return output_dir

def create_data_transforms():
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),  # 调整裁剪范围
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(p=0.1),
            transforms.RandomRotation(30),  # 增加旋转角度
            transforms.ColorJitter(
                brightness=0.3,  # 增加亮度调整范围
                contrast=0.3,    # 增加对比度调整范围
                saturation=0.3,  # 增加饱和度调整范围
                hue=0.1          # 添加色调调整
            ),
            transforms.RandomGrayscale(p=0.1),
            transforms.RandomAffine(  # 仿射变换
                degrees=0,
                translate=(0.1, 0.1),  # 平移
                scale=(0.9, 1.1),      # 缩放
                shear=10               # 剪切
            ),
            transforms.RandomPerspective(distortion_scale=0.2, p=0.2),  # 透视变换
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # 高斯模糊
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def create_data_loaders(data_dir):
    """创建数据加载器"""
    data_transforms = create_data_transforms()
    
    # 创建数据集
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'val', 'test']}
    
    # 创建数据加载器
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                 shuffle=True if x == 'train' else False,
                                 num_workers=NUM_WORKERS)
                   for x in ['train', 'val', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes
    
    return dataloaders, dataset_sizes, class_names

def initialize_model(model_name, num_classes, feature_extract=True):
    """初始化预训练模型"""
    model_info = MODELS[model_name]
    model_ft = model_info["model"](pretrained=model_info["pretrained"])
    
    # 冻结预训练模型的参数
    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False
    
    # 修改分类器
    if model_name == "efficientnet-b0":
        model_ft.classifier[1] = nn.Linear(model_info["feature_size"], num_classes)
    
    elif model_name == "mobilenet_v3":
        model_ft.classifier[3] = nn.Linear(model_info["feature_size"], num_classes)
    
    elif model_name == "resnet50":
        model_ft.fc = nn.Linear(model_info["feature_size"], num_classes)
    
    elif model_name == "vgg16":
        model_ft.classifier[6] = nn.Linear(model_info["feature_size"], num_classes)
    
    return model_ft

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, model_name="model"):
    """训练模型"""
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # 记录训练历史
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 训练模式
            else:
                model.eval()   # 评估模式
            
            running_loss = 0.0
            running_corrects = 0
            
            # 迭代数据
            progress_bar = tqdm(enumerate(dataloaders[phase]), total=len(dataloaders[phase]))
            for i, (inputs, labels) in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 零梯度
                optimizer.zero_grad()
                
                # 前向传播
                # 只有在训练时才跟踪历史
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # 只有在训练阶段才进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                # 更新进度条
                progress_bar.set_description(f"{phase} Loss: {running_loss/(i*BATCH_SIZE+inputs.size(0)):.4f} Acc: {running_corrects/(i*BATCH_SIZE+inputs.size(0)):.4f}")
            
            if phase == 'train' and scheduler:
                scheduler.step()
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # 记录历史
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            
            # 深拷贝模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), f"{model_name}_best_model.pth")
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model, history

def evaluate_model(model, dataloader, class_names, model_name):
    """评估模型"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # 打印分类报告
    print(f"\n{classification_report(all_labels, all_preds, target_names=class_names)}")
    
    # 计算混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    plot_confusion_matrix(cm, class_names, f"{model_name}_confusion_matrix.png")
    
    return all_preds, all_labels

def plot_confusion_matrix(cm, classes, save_path, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """绘制混淆矩阵"""
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(save_path)
    plt.close()

def plot_training_curves(histories, save_path='training_curves.png'):
    """绘制多个模型的训练曲线"""
    plt.figure(figsize=(12, 10))
    
    # 绘制准确率曲线
    plt.subplot(2, 1, 1)
    for model_name, history in histories.items():
        plt.plot(history['train_acc'], label=f'{model_name} Train')
        plt.plot(history['val_acc'], label=f'{model_name} Val')
    
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    # 绘制损失曲线
    plt.subplot(2, 1, 2)
    for model_name, history in histories.items():
        plt.plot(history['train_loss'], label=f'{model_name} Train')
        plt.plot(history['val_loss'], label=f'{model_name} Val')
    
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def main():
    """主函数"""
    # 划分数据集
    split_dir = split_dataset(DATA_DIR, OUTPUT_DIR, SPLIT_RATIO)
    
    # 创建数据加载器
    dataloaders, dataset_sizes, class_names = create_data_loaders(split_dir)
    print(f"Classes: {class_names}")
    num_classes = len(class_names)
    
    all_histories = {}
    
    # 训练所有模型
    for model_name in MODELS.keys():
        print(f"\n===== Training {model_name} model =====")
        
        # 初始化模型
        model_ft = initialize_model(model_name, num_classes, feature_extract=False)
        model_ft = model_ft.to(device)
        
        # 设置损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        
        # 只优化最后一层的参数
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
        
        # 学习率调度器
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
        
        # 训练模型
        model_ft, history = train_model(
            model_ft, dataloaders, criterion, optimizer_ft, exp_lr_scheduler,
            num_epochs=EPOCHS, model_name=model_name
        )
        
        # 保存最终模型
        torch.save(model_ft.state_dict(), f"{model_name}_final_model.pth")
        
        # 评估模型
        print(f"\nEvaluating {model_name} model:")
        evaluate_model(model_ft, dataloaders['test'], class_names, model_name)
        
        # 保存训练历史
        all_histories[model_name] = history
    
    # 绘制所有模型的训练曲线
    plot_training_curves(all_histories, save_path='all_models_training_curves.png')
    
    print("\n===== Training completed for all models =====")

if __name__ == "__main__":
    main()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms
import time
import copy
import shutil
from tqdm import tqdm

# 设置随机种子确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 配置参数
DATA_DIR = r"D:\homework\论文\论文\project\dataset\archive\Rice_Leaf_AUG\Rice_Leaf_AUG"  # 原始数据集路径
OUTPUT_DIR = "dataset_split"  # 划分后的数据集保存路径
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 1
NUM_WORKERS = 4  # 数据加载的线程数
SPLIT_RATIO = [0.7, 0.15, 0.15]  # 训练集、验证集、测试集比例

# 模型配置
MODELS = {
    #     "resnet50": {
    #     "model": models.resnet50,
    #     "pretrained": True,
    #     "feature_size": 2048  # ResNet50的特征维度
    # },
    # "vgg16": {
    #     "model": models.vgg16,
    #     "pretrained": True,
    #     "feature_size": 512 * 7 * 7  # VGG16的特征维度
    # },
    # "efficientnet-b0": {
    #     "model": models.efficientnet_b0,
    #     "pretrained": True,
    #     "feature_size": 1280  # EfficientNet-B0的特征维度
    # },
    "mobilenet_v3": {
        "model": models.mobilenet_v3_large,
        "pretrained": True,
        "feature_size": 960  # MobileNetV3的特征维度
    },

}

# 检查GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

def split_dataset(data_dir, output_dir, split_ratio):
    """将数据集划分为训练集、验证集和测试集"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    subdirs = [f.name for f in os.scandir(data_dir) if f.is_dir()]
    
    for subdir in subdirs:
        print(f"Processing class: {subdir}")
        class_dir = os.path.join(data_dir, subdir)
        files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # 划分数据集
        train_files, test_files = train_test_split(files, test_size=split_ratio[2], random_state=42)
        train_files, val_files = train_test_split(train_files, test_size=split_ratio[1]/(split_ratio[0]+split_ratio[1]), random_state=42)
        
        # 创建输出目录
        for split in ["train", "val", "test"]:
            split_dir = os.path.join(output_dir, split, subdir)
            os.makedirs(split_dir, exist_ok=True)
        
        # 复制文件
        def copy_files(files_list, split_name):
            for file_name in tqdm(files_list, desc=f"Copying {split_name} files"):
                src = os.path.join(class_dir, file_name)
                dst = os.path.join(output_dir, split_name, subdir, file_name)
                shutil.copy(src, dst)
        
        copy_files(train_files, "train")
        copy_files(val_files, "val")
        copy_files(test_files, "test")
    
    print(f"Dataset split completed. Saved to {output_dir}")
    return output_dir

def create_data_transforms():
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),  # 调整裁剪范围
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(p=0.1),
            transforms.RandomRotation(30),  # 增加旋转角度
            transforms.ColorJitter(
                brightness=0.3,  # 增加亮度调整范围
                contrast=0.3,    # 增加对比度调整范围
                saturation=0.3,  # 增加饱和度调整范围
                hue=0.1          # 添加色调调整
            ),
            transforms.RandomGrayscale(p=0.1),
            transforms.RandomAffine(  # 仿射变换
                degrees=0,
                translate=(0.1, 0.1),  # 平移
                scale=(0.9, 1.1),      # 缩放
                shear=10               # 剪切
            ),
            transforms.RandomPerspective(distortion_scale=0.2, p=0.2),  # 透视变换
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # 高斯模糊
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def create_data_loaders(data_dir):
    """创建数据加载器"""
    data_transforms = create_data_transforms()
    
    # 创建数据集
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'val', 'test']}
    
    # 创建数据加载器
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                 shuffle=True if x == 'train' else False,
                                 num_workers=NUM_WORKERS)
                   for x in ['train', 'val', 'test']}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes
    
    return dataloaders, dataset_sizes, class_names

def initialize_model(model_name, num_classes, feature_extract=True):
    """初始化预训练模型"""
    model_info = MODELS[model_name]
    model_ft = model_info["model"](pretrained=model_info["pretrained"])
    
    # 冻结预训练模型的参数
    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False
    
    # 修改分类器
    if model_name == "efficientnet-b0":
        model_ft.classifier[1] = nn.Linear(model_info["feature_size"], num_classes)
    
    elif model_name == "mobilenet_v3":
        model_ft.classifier[3] = nn.Linear(model_info["feature_size"], num_classes)
    
    elif model_name == "resnet50":
        model_ft.fc = nn.Linear(model_info["feature_size"], num_classes)
    
    elif model_name == "vgg16":
        model_ft.classifier[6] = nn.Linear(model_info["feature_size"], num_classes)
    
    return model_ft

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, model_name="model"):
    """训练模型"""
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # 记录训练历史
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)
        
        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 训练模式
            else:
                model.eval()   # 评估模式
            
            running_loss = 0.0
            running_corrects = 0
            
            # 迭代数据
            progress_bar = tqdm(enumerate(dataloaders[phase]), total=len(dataloaders[phase]))
            for i, (inputs, labels) in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 零梯度
                optimizer.zero_grad()
                
                # 前向传播
                # 只有在训练时才跟踪历史
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # 只有在训练阶段才进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                # 更新进度条
                progress_bar.set_description(f"{phase} Loss: {running_loss/(i*BATCH_SIZE+inputs.size(0)):.4f} Acc: {running_corrects/(i*BATCH_SIZE+inputs.size(0)):.4f}")
            
            if phase == 'train' and scheduler:
                scheduler.step()
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # 记录历史
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())
            
            # 深拷贝模型
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), f"{model_name}_best_model.pth")
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model, history

def evaluate_model(model, dataloader, class_names, model_name):
    """评估模型"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # 打印分类报告
    print(f"\n{classification_report(all_labels, all_preds, target_names=class_names)}")
    
    # 计算混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    plot_confusion_matrix(cm, class_names, f"{model_name}_confusion_matrix.png")
    
    return all_preds, all_labels

def plot_confusion_matrix(cm, classes, save_path, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """绘制混淆矩阵"""
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(save_path)
    plt.close()

def plot_training_curves(histories, save_path='training_curves.png'):
    """绘制多个模型的训练曲线"""
    plt.figure(figsize=(12, 10))
    
    # 绘制准确率曲线
    plt.subplot(2, 1, 1)
    for model_name, history in histories.items():
        plt.plot(history['train_acc'], label=f'{model_name} Train')
        plt.plot(history['val_acc'], label=f'{model_name} Val')
    
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    # 绘制损失曲线
    plt.subplot(2, 1, 2)
    for model_name, history in histories.items():
        plt.plot(history['train_loss'], label=f'{model_name} Train')
        plt.plot(history['val_loss'], label=f'{model_name} Val')
    
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def main():
    """主函数"""
    # 划分数据集
    split_dir = split_dataset(DATA_DIR, OUTPUT_DIR, SPLIT_RATIO)
    
    # 创建数据加载器
    dataloaders, dataset_sizes, class_names = create_data_loaders(split_dir)
    print(f"Classes: {class_names}")
    num_classes = len(class_names)
    
    all_histories = {}
    
    # 训练所有模型
    for model_name in MODELS.keys():
        print(f"\n===== Training {model_name} model =====")
        
        # 初始化模型
        model_ft = initialize_model(model_name, num_classes, feature_extract=False)
        model_ft = model_ft.to(device)
        
        # 设置损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        
        # 只优化最后一层的参数
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
        
        # 学习率调度器
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
        
        # 训练模型
        model_ft, history = train_model(
            model_ft, dataloaders, criterion, optimizer_ft, exp_lr_scheduler,
            num_epochs=EPOCHS, model_name=model_name
        )
        
        # 保存最终模型
        torch.save(model_ft.state_dict(), f"{model_name}_final_model.pth")
        
        # 评估模型
        print(f"\nEvaluating {model_name} model:")
        evaluate_model(model_ft, dataloaders['test'], class_names, model_name)
        
        # 保存训练历史
        all_histories[model_name] = history
    
    # 绘制所有模型的训练曲线
    plot_training_curves(all_histories, save_path='all_models_training_curves.png')
    
    print("\n===== Training completed for all models =====")

if __name__ == "__main__":
    main()